In [2]:
import os
import pandas as pd
from tqdm import tqdm

tqdm.pandas()
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI

In [3]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://kmt-oai-uk-37.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "dbbe0f299cd44d38ba566abf5202f52d"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"

### 데이터 로드

In [4]:
df = pd.read_csv("../data/poi_review.csv")

### 데이터 필터링

In [5]:
def get_categories(fullpath):
    categories = []
    splited = list(map(str.strip, fullpath.split(">")))
    for s in splited:
        s = s.strip()
        if s == "":
            continue

        categories_ = list(map(str.strip, s.split(",")))

        categories.extend(categories_)

    return categories

In [6]:
filtered_df = df[df['reviews'].apply(lambda x: isinstance(x, str) and str(x).strip() != '')]
filtered_df["scores"] = filtered_df["scores"].apply(eval)
filtered_df = filtered_df[filtered_df["scores"].apply(sum) / filtered_df["scores"].apply(len) >= 3.5]
filtered_df = filtered_df[filtered_df["reviews"].apply(len) > 50]
filtered_df["reviews"] = filtered_df["reviews"].apply(lambda x : x[1:-1])
filtered_df["categories"] = filtered_df["fullpath"].apply(get_categories)

In [7]:
filtered_df

,poi_id,poi_name,visit_count,hcode,hname1,hname2,hname3,cate_id,fullpath,catename,...,landmarkyn,name1,name2,name3,name4,name5,time_list,scores,reviews,categories
5,9107967,서적백화점,108,1168067000,서울특별시,강남구,개포2동,212.0,"문화,예술>도서>서점",서점,...,N,"문화,예술",도서,서점,NaN,NaN,"[{""periodEtc"":null,""periodName"":""영업기간"",""timeLi...","[4, 5, 1, 5, 5, 5]",대치 학원가에서 사용하는 모든 교재는 여기에 다 있어요. 책 찾아달라고 말씀 드리면...,"[문화, 예술, 도서, 서점]"
7,14542282,알라딘중고서점,136,1111061500,서울특별시,종로구,종로1.2.3.4가동,880.0,"문화,예술>도서>서점>중고서점",중고서점,...,N,"문화,예술",도서,서점,중고서점,NaN,"[{""periodEtc"":null,""periodName"":""영업기간"",""timeLi...","[5, 4, 2, 5, 3, 4, 5, 5, 5, 5, 1, 4, 5, 1, 2, 1]",책 판매하러 갔는데 직원분들 엄청 친절하세요. 남/여 두분 다 너무 친절했어요. 주...,"[문화, 예술, 도서, 서점, 중고서점]"
8,434486987,알라딘중고서점,75,1130559500,서울특별시,강북구,번1동,880.0,"문화,예술>도서>서점>중고서점",중고서점,...,N,"문화,예술",도서,서점,중고서점,NaN,"[{""periodEtc"":null,""periodName"":""영업기간"",""timeLi...","[5, 5, 5, 5, 5, 5, 4, 1]",",전에 카페도 있었을 때가 더 좋았지만!,좋아요,친절하고 깔금하고 왠만한 책들은 다...","[문화, 예술, 도서, 서점, 중고서점]"
10,19511091,알라딘중고서점,46,1162064500,서울특별시,관악구,서원동,880.0,"문화,예술>도서>서점>중고서점",중고서점,...,N,"문화,예술",도서,서점,중고서점,NaN,"[{""periodEtc"":null,""periodName"":""영업기간"",""timeLi...","[5, 5, 1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 2, 4, 4]","책 많고 직원분이 친절하십니다,매장이 크고 깨끗해서 좋아보였음.,알라딘 온라인으로 ...","[문화, 예술, 도서, 서점, 중고서점]"
11,1794507908,서울책보고,114,1171071000,서울특별시,송파구,잠실6동,880.0,"문화,예술>도서>서점>중고서점",중고서점,...,N,"문화,예술",도서,서점,중고서점,NaN,"[{""periodEtc"":null,""periodName"":""영업기간"",""timeLi...","[1, 5, 5, 5, 5, 5, 1, 5, 5, 5, 1, 4, 5, 5, 3, 1]","직원들은 무례합니다,책이 다양하게 있진 않지만 조용하고 분위기가 좋아요 인스타그램에...","[문화, 예술, 도서, 서점, 중고서점]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4449,8595348,지호한방삼계탕,40,1121584700,서울특별시,광진구,자양4동,2933.0,"음식점>한식>육류,고기>닭요리>지호한방삼계탕",지호한방삼계탕,...,N,음식점,한식,"육류,고기",닭요리,지호한방삼계탕,"[{""periodName"":""영업기간"",""periodEtc"":null,""timeLi...","[5, 3, 5, 5, 5, 3, 5, 1]",",아주머니들이 너무 바쁘셔서 자리도 잘 정리가 안되어 있었어요 ㅠㅠ 맛은 쏘쏘..,...","[음식점, 한식, 육류, 고기, 닭요리, 지호한방삼계탕]"
4452,10577180,엉터리생고기,75,1135064000,서울특별시,노원구,상계2동,3026.0,"음식점>한식>육류,고기>엉터리생고기",엉터리생고기,...,N,음식점,한식,"육류,고기",엉터리생고기,NaN,"[{""periodName"":""영업기간"",""periodEtc"":null,""timeLi...","[5, 5, 5]","한 번 날잡고 고기먹기 좋습니다서비스 훌륭하구요번창하세요!,항상 맛있습니다!! 사장...","[음식점, 한식, 육류, 고기, 엉터리생고기]"
4453,10635428,옛골토성,125,1165066000,서울특별시,서초구,내곡동,1579.0,"음식점>한식>육류,고기>옛골토성",옛골토성,...,N,음식점,한식,"육류,고기",옛골토성,NaN,"[{""periodName"":""영업기간"",""periodEtc"":null,""timeLi...","[1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",와... 사장님 진짜 싸가지 없으시네요카드를 낚아채는 식당 주인은 처음밥 먹고 기분...,"[음식점, 한식, 육류, 고기, 옛골토성]"
4459,967792291,족발신선생,160,1150060300,서울특별시,강서구,가양1동,25408.0,"음식점>한식>육류,고기>족발,보쌈>족발신선생",족발신선생,...,N,음식점,한식,"육류,고기","족발,보쌈",족발신선생,NaN,"[5, 5, 1, 5, 5]",다른 족발집 웨이팅하다가 너무 오래걸려서 여기로 왔어요.굉장히 만족스럽네요 ㅋㅋ 서...,"[음식점, 한식, 육류, 고기, 족발, 보쌈, 족발신선생]"


In [8]:
food_shop_df =  filtered_df[filtered_df["categories"].progress_apply(lambda x : "음식점" in x)]

100%|██████████| 1949/1949 [00:00<00:00, 602676.09it/s]


In [9]:
food_shop_df

,poi_id,poi_name,visit_count,hcode,hname1,hname2,hname3,cate_id,fullpath,catename,...,landmarkyn,name1,name2,name3,name4,name5,time_list,scores,reviews,categories
1838,2068929425,달콤왕가탕후루,87,1120057000,서울특별시,성동구,행당2동,46.0,음식점>간식,간식,...,N,음식점,간식,NaN,NaN,NaN,"[{""periodName"":""영업기간"",""periodEtc"":null,""timeLi...","[5, 5, 4, 1]","정말 맛있습니다. 또 방문하겠읍니다.,가성비 짱짱!!! 우리동네 맛집입니다^^매일 ...","[음식점, 간식]"
1839,1649765943,랜디스도넛,145,1144071000,서울특별시,마포구,연남동,791.0,음식점>간식>도넛,도넛,...,N,음식점,간식,도넛,NaN,NaN,"[{""periodName"":""영업기간"",""periodEtc"":null,""timeLi...","[2, 3, 3, 2, 5, 5, 3, 5, 3, 5, 3]",맛이 없는 것보다 더 별로인건 맛이 변하는거 같아요. 오픈 했을 때부터 꾸준히 한 ...,"[음식점, 간식, 도넛]"
1840,802213706,올드페리도넛,75,1117068500,서울특별시,용산구,한남동,791.0,음식점>간식>도넛,도넛,...,N,음식점,간식,도넛,NaN,NaN,"[{""periodName"":""영업기간"",""periodEtc"":null,""timeLi...","[3, 1, 5, 4, 5, 5, 5, 5, 4, 5, 4, 4, 1, 5, 4, ...","조금 헤비할 수 있지만 맛있는 도넛,테이크아웃만 가능해서 가져왔는데 집에서 열어보니...","[음식점, 간식, 도넛]"
1842,17004790,구름떡,138,1165062100,서울특별시,서초구,방배4동,1010.0,"음식점>간식>떡,한과","떡,한과",...,N,음식점,간식,"떡,한과",NaN,NaN,"[{""periodName"":""영업기간"",""periodEtc"":null,""timeLi...","[5, 5, 5, 4, 5, 5, 5, 4, 3, 2, 5, 5, 5, 5, 5, 4]","흑임자떡 정말 맛있어요,흑임자떡 도랏음 그냥 재질미침… ㅜㅜ내가 태어나서 먹은 떡중...","[음식점, 간식, 떡, 한과]"
1843,18310200,백년화편,100,1174066000,서울특별시,강동구,성내3동,1010.0,"음식점>간식>떡,한과","떡,한과",...,N,음식점,간식,"떡,한과",NaN,NaN,"[{""periodName"":""영업기간"",""periodEtc"":null,""timeLi...","[5, 5, 4, 5, 5, 3, 2, 5, 4, 3, 5, 5, 5, 5, 2, ...","밥알떡 너무 맛나요,맛있어요 밥알 찹살떡 최고예요~~~,매장에서 먹을순없고 매장에서...","[음식점, 간식, 떡, 한과]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4449,8595348,지호한방삼계탕,40,1121584700,서울특별시,광진구,자양4동,2933.0,"음식점>한식>육류,고기>닭요리>지호한방삼계탕",지호한방삼계탕,...,N,음식점,한식,"육류,고기",닭요리,지호한방삼계탕,"[{""periodName"":""영업기간"",""periodEtc"":null,""timeLi...","[5, 3, 5, 5, 5, 3, 5, 1]",",아주머니들이 너무 바쁘셔서 자리도 잘 정리가 안되어 있었어요 ㅠㅠ 맛은 쏘쏘..,...","[음식점, 한식, 육류, 고기, 닭요리, 지호한방삼계탕]"
4452,10577180,엉터리생고기,75,1135064000,서울특별시,노원구,상계2동,3026.0,"음식점>한식>육류,고기>엉터리생고기",엉터리생고기,...,N,음식점,한식,"육류,고기",엉터리생고기,NaN,"[{""periodName"":""영업기간"",""periodEtc"":null,""timeLi...","[5, 5, 5]","한 번 날잡고 고기먹기 좋습니다서비스 훌륭하구요번창하세요!,항상 맛있습니다!! 사장...","[음식점, 한식, 육류, 고기, 엉터리생고기]"
4453,10635428,옛골토성,125,1165066000,서울특별시,서초구,내곡동,1579.0,"음식점>한식>육류,고기>옛골토성",옛골토성,...,N,음식점,한식,"육류,고기",옛골토성,NaN,"[{""periodName"":""영업기간"",""periodEtc"":null,""timeLi...","[1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",와... 사장님 진짜 싸가지 없으시네요카드를 낚아채는 식당 주인은 처음밥 먹고 기분...,"[음식점, 한식, 육류, 고기, 옛골토성]"
4459,967792291,족발신선생,160,1150060300,서울특별시,강서구,가양1동,25408.0,"음식점>한식>육류,고기>족발,보쌈>족발신선생",족발신선생,...,N,음식점,한식,"육류,고기","족발,보쌈",족발신선생,NaN,"[5, 5, 1, 5, 5]",다른 족발집 웨이팅하다가 너무 오래걸려서 여기로 왔어요.굉장히 만족스럽네요 ㅋㅋ 서...,"[음식점, 한식, 육류, 고기, 족발, 보쌈, 족발신선생]"


In [10]:
from api import km_api
food_shop_df['coord']=food_shop_df.progress_apply(km_api.get_detail_coord_with_hcode, axis=1)


100%|██████████| 1248/1248 [02:34<00:00,  8.07it/s]
/var/folders/n5/wylv0r_d41lcmb6fxb5j2s9h0000gn/T/ipykernel_65745/4133684501.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_shop_df['coord']=food_shop_df.progress_apply(km_api.get_detail_coord_with_hcode, axis=1)


food_shop_df.to_csv('../data/poi_coord.csv', index=True)

In [12]:
food_shop_df.to_csv('../data/poi_coord_2.csv', index=True)

In [15]:
food_shop_df['catename'].unique().tolist()

['간식',
 '도넛',
 '떡,한과',
 '아이스크림',
 '제과,베이커리',
 '기사식당',
 '분식',
 '떡볶이',
 '뷔페',
 '고기뷔페',
 '한식뷔페',
 '해산물뷔페',
 '샐러드',
 '샤브샤브',
 '술집',
 '실내포장마차',
 '와인바',
 '일본식주점',
 '칵테일바',
 '호프,요리주점',
 '아시아음식',
 '동남아음식',
 '베트남음식',
 '태국음식',
 '양식',
 '멕시칸,브라질',
 '스테이크,립',
 '이탈리안',
 '피자',
 '바닷가재',
 '햄버거',
 '일식',
 '돈까스,우동',
 '일본식라면',
 '일식집',
 '참치회',
 '초밥,롤',
 '중식',
 '양꼬치',
 '중국요리',
 '치킨',
 '카페',
 '다방',
 '커피전문점',
 '테마카페',
 '갤러리카페',
 '디저트카페',
 '무인카페',
 '북카페',
 '애견카페',
 '패밀리레스토랑',
 '패스트푸드',
 '샌드위치',
 '푸드코트',
 '퓨전요리',
 '퓨전일식',
 '퓨전중식',
 '퓨전한식',
 '한식',
 '감자탕',
 '곰탕',
 '국밥',
 '국수',
 '칼국수',
 '냉면',
 '두부전문점',
 '사철탕,영양탕',
 '설렁탕',
 '수제비',
 '순대',
 '쌈밥',
 '육류,고기',
 '갈비',
 '곱창,막창',
 '닭요리',
 '삼계탕',
 '불고기,두루치기',
 '삼겹살',
 '오리',
 '족발,보쌈',
 '찌개,전골',
 '한정식',
 '해물,생선',
 '게,대게',
 '매운탕,해물탕',
 '복어',
 '아구',
 '장어',
 '조개',
 '추어',
 '회',
 '해장국',
 '김영모과자점',
 '아티제',
 '코코호도',
 '파리바게뜨',
 '등촌샤브칼국수',
 '채선당',
 '금별맥주',
 '범맥주',
 '미분당',
 '도미노피자',
 '피자마루',
 '피자스쿨',
 '역전우동0410',
 '깐부치킨',
 '노랑통닭',
 '바른치킨',
 '치킨마루',
 '페리카나',
 '오설록 티하우스',
 '더벤티',
 '빽다방',
 

In [9]:
food_shop_df['coord'].progress_apply(km_api.get_distance, coord2 = (37.5395997372027,127.003009774096))

100%|██████████| 1248/1248 [00:00<00:00, 347806.74it/s]


1838     3211.535243
1839     7387.893207
1840        0.000000
1842     5464.033509
1843    11563.464931
            ...     
4449    10000.000000
4452    10000.000000
4453    10000.000000
4459    10000.000000
4474    10000.000000
Name: coord, Length: 1248, dtype: float64

### 모델 선언

In [16]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

review_summarize_system_template = """
너는 "리뷰묶음"을 받아서 리뷰를 요약해주는 AI모델이야.
너는 입력으로 가게의 "리뷰묶음" 정보를 입력받아서 해당 리뷰를 요약해주는 문장을 생성해야 해.
이때 리뷰에서 부정적인 내용, 가게에 좋지 않은 내용, 기분이 나쁜 내용들은 절대로 리뷰 내용에 포함하지 말아야 해.
또한 "리뷰묶음"에 포함되어 있지 않았해|던 정보는 포함되면 안 돼.
그리고 추가적으로 리뷰 요약과 리뷰 내용을 통해 다음과 같은 내용 중 한 가지를 유츄해 선택해야 해
- "사람들이 주로 방문하는 시간대", "사람들이 주로 같이 방문하는 인원", "식당의 분위기", "식당 메뉴"
이때 주어진 리뷰를 기반으로 위의 정보들을 알 수 없는 경우에는 "알 수 없음"으로 응답해야 해
"""

review_summarize_question_example1 = """
[입력] :
- 가게 이름 : 달콤왕가탕후루
- 가게 위치 : 서울특별시 성동구 행당2동
- 카테고리 : [음식점, 간식]
- 리뷰 묶음 : 정말 맛있습니다. 또 방문하겠읍니다.,가성비 짱짱!!! 우리동네 맛집입니다^^매일 가고 싶네요 ^^,귤탕후루, 체리탕후루 중 귤탕후루가 맛있었당!,딸기탕후루 무슨 일입니까 진짜 ㅋㅋ이렇게 파실거면 가격을 내려야죠. 장난하십니까.딸기가 미니어쳐 수준  블루베리 크기만해요.너무나도 실망스럽습니다.
"""

review_summarize_answer_example1 = """
- 리뷰 요약 : 달콤왕가탕후루는 서울특별시 성동구에 위치하며 다양한 탕후루를 파는 가게이며, 동네에서 싸고 맛있는 가게로 소문이 나있습니다. 특히 귤탕후루가 맛있다고 합니다.
- 시간대 : 알 수 없음
- 인원 : 알 수 없음
- 분위기 : 알 수 없음
- 식당 메뉴 : 귤탕후루, 체리탕후루
"""

review_summarize_question_example2 = """
[입력] :
- 가게 이름 : 족발신선생
- 가게 위치 : 서울특별시 강서구 가양1동
- 카테고리 : [음식점, 한식, 육류, 고기, 족발, 보쌈, 족발신선생]
- 리뷰 묶음 : 다른 족발집 웨이팅하다가 너무 오래걸려서 여기로 왔어요.굉장히 만족스럽네요 ㅋㅋ 서비스라던가 메뉴 퀄리티 모두 좋습니다. 잘 먹고 가용 🫶,족발 정말 맛있어요.,여자직원들 싸가지없음,족발 맛있고,, 진짜 양 미텼고.. 서비스로 주는 어리굴젓 진짜 너무 맛있음 ㅠ 살코기 많은것 보다 껍질이 더 많은 느낌이라 쫄깃한 껍질을 더 좋아하면 가셔욤!! 사이드디쉬로 나온 짬뽕 순두부도 다 먹지 못할정도로 양이 많았어요~~강추~~,
"""

review_summarize_answer_example2 = """
- 리뷰 요약 : 족발신선생님 서울특별시 강서구 가양1동에 위치하는 한식 음식점이며 주로 족발과 보쌈을 판매하는 식당입니다. 다른 매장에 비해 웨이팅 없이 빠르게 입장할 수 있으며 서비스 및 퀄리티가 매우 좋습니다. 음식의 양 또한 많으며 특시 서비스로 주는 어리굴젓이 맛있다고 합니다.
- 시간대 : 알 수 없음
- 인원 : 알 수 없음
- 분위기 : 알 수 없음
- 식당 메뉴 : 족발, 짬뽕 순두부, 보쌈
"""

user_input = """
[입력] :
- 가게 이름 : {store_name}
- 가게 위치 : {store_location}
- 카테고리 : {category}
- 리뷰 묶음 : {reviews}
"""

In [17]:
sample = food_shop_df.loc[2916]
sample

poi_id                                                1298505567
poi_name                                                    섭생정식
visit_count                                                   42
hcode                                                 1132051500
hname1                                                     서울특별시
hname2                                                       도봉구
hname3                                                       창5동
cate_id                                                    124.0
fullpath                                                  음식점>한식
catename                                                      한식
catetype                                                       C
apicatename                                                  음식점
adultflag                                                      N
landmarkyn                                                     N
name1                                                        음식점
name2                    

In [18]:
chat = AzureChatOpenAI(azure_deployment="gpt-4")

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=review_summarize_system_template),
        HumanMessage(content=review_summarize_question_example1),
        AIMessage(content=review_summarize_answer_example1),
        HumanMessage(content=review_summarize_question_example2),
        AIMessage(content=review_summarize_answer_example2),
        HumanMessage(content=user_input.format(store_name=sample["poi_name"], store_location=" ".join(
            [sample["hname1"], sample["hname2"], sample["hname3"]]), category=sample["categories"],
                                               reviews=sample["reviews"])),
    ]
)

chain = prompt | chat

/Users/frankie.gg/Documents/GitHub/kamo-ai-hackerton/venv/lib/python3.9/site-packages/langchain/chat_models/azure_openai.py:155: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://kmt-oai-uk-37.openai.azure.com/ to https://kmt-oai-uk-37.openai.azure.com/openai.
  warnings.warn(
/Users/frankie.gg/Documents/GitHub/kamo-ai-hackerton/venv/lib/python3.9/site-packages/langchain/chat_models/azure_openai.py:162: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/Users/frankie.gg/Documents/GitHub/kamo-ai-hackerton/venv/lib/python3.9/site-packages/langchain/chat_models/azure_openai.py:170: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_

In [19]:
chain.invoke({})

AIMessage(content='- 리뷰 요약 : 섭생정식은 서울특별시 도봉구 창5동에 위치한 한식 음식점으로, 신선한 해산물을 사용한 요리가 특징입니다. 해물탕, 산낙지 연포탕, 파전 등의 메뉴가 인기 있으며, 조미료를 사용하지 않고 건강한 맛을 내는 것으로 평가받고 있습니다. 손님들에게 친절하게 설명해 주며, 해산물의 신선도와 훌륭한 퀄리티에도 불구하고 가격 대비 만족도가 높은 것으로 보입니다.\n- 시간대 : 알 수 없음\n- 인원 : 알 수 없음\n- 분위기 : 알 수 없음\n- 식당 메뉴 : 해물탕, 산낙지 연포탕, 파전')